Create a Hive enabled spark session, with a persisted derby metastore. Start Hive Thrift server to enable JDBC connections.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SvnLocalSpark")\
    .enableHiveSupport()\
    .getOrCreate()

sc = spark.sparkContext

print(f"spark {spark.version} {sc.uiWebUrl}")

spark 3.5.4 http://DESKTOP-4GOMK6M:4042


In [2]:
spark.sql("CREATE SCHEMA IF NOT EXISTS landing")
# create external table
spark.catalog.getDatabase("landing")
spark.sql("DROP TABLE IF EXISTS landing.properties")
spark.catalog.createTable(
    tableName = "landing.commercial_properties",
    source = "csv",
    description = "property values",
    header="true", delimiter=",", path="../../../resources/sourcedata/commercial_property_snapshots_100_M39.csv", inferSchema="true")

DataFrame[date: date, property_id: string, street: string, street_number: int, city: string, zip_code: int, category: string, property_value: double, energy_label: string]

In [3]:
for db in spark.catalog.listDatabases():
    print(f"database: {db.name}")
    for tbl in spark.catalog.listTables(db.name):
        print(f"table:{tbl.name}")
print(f"current database:{spark.catalog.currentDatabase()}")

database: default
database: landing
table:commercial_properties
current database:default


In [ ]:
# Start Hive Thrift Server
java_args = sc._gateway.new_array(sc._gateway.jvm.java.lang.String,2)
java_args[0]="dummy"
java_args[1]="false"

sc._jvm.org.apache.spark.sql.hive.thriftserver.HiveThriftServer2.main(java_args)